<a href="https://colab.research.google.com/github/FabioMMaia/CelebNet/blob/main/Modeling_HOG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import cv2
from PIL import Image
from skimage import data, exposure
from skimage.feature import hog,local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from numpy.random import seed
import pickle
from tqdm import tqdm

import pytz
from datetime import datetime
from collections import Counter
import math

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

In [148]:
!pip install unidecode

def norm_text(txt):
  from unidecode import unidecode
  return unidecode(txt).replace(' ','_').lower()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
config_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/config/'
model_save_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/model/'
erro_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/error/'
log_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/log/'

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Modelos_Implementações')

In [347]:
import MLP as mlp # multilayer perceptron
import Multinomial_LogReg # regressão logística com alpha variavel (bissecao)
import R_LOG as r_log # regressão logística com alpha fixo
import SVM_OVR # SVM one-versus-rest
import ANN

In [403]:
import importlib

importlib.reload(ANN)
importlib.reload(mlp)
importlib.reload(Multinomial_LogReg)
importlib.reload(r_log)
importlib.reload(SVM_OVR)
importlib.reload(ANN)

<module 'ANN' from '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Modelos_Implementações/ANN.py'>

In [6]:
class logfile():

  def __init__(self, diretorio):
    self.diretorio=diretorio

  def start(self, tag, use_dateref=True):
    if use_dateref:
      tag = tag + '_' + datetime.now(pytz.timezone('America/Bahia')).strftime("%Y%m%d")
    self.save_file = open(self.diretorio +  tag + '.txt', 'w')
    data_atual = datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")
    print('Execução em:', data_atual, file=self.save_file)
    
  def end(self):
    data_atual = datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")
    print('Fim de Execução em:', data_atual, file=self.save_file)
    self.save_file.close()

# Carregando Dataset - HOG

In [57]:
n_classes=3
ref = 'HOG' 

tag= 'hog' + '_'+ str(n_classes) + '_classes'
with open(r'/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Data/X_y_' + tag +'.obj', "rb") as f:
    X,y = pickle.load(f)

In [58]:
print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (101, 4608)
y shape: (101,)


In [59]:
# transformar categorias em one-hot-encoding: Saída
saida = y.max()+1
print("Quantidade de pessoas:",saida)
target = keras.utils.to_categorical(y, saida)

Quantidade de pessoas: 3


# Train x Test

In [60]:
def train_test_separation(X, y):
  x_train, x_test, y_train_n, y_test_n = train_test_split(X, y, test_size=0.30, random_state=42,stratify=y)

  y_train = target[y.index.isin(y_train_n.index).flatten(),:]
  y_test= target[y.index.isin(y_test_n.index).flatten(),:]

  assert y_train.shape[0]== y_train_n.shape[0]
  assert y_test.shape[0]== y_test_n.shape[0]

  print('Shape',x_train.shape, x_test.shape, y_train.shape, y_test.shape,  y_train_n.shape, y_test_n.shape)

  return x_train, x_test, y_train_n, y_test_n, y_train, y_test

In [61]:
def apply_PCA(n_pca, x_train,x_test):
  pca = PCA(n_components=n_pca, random_state=42)
  pca.fit(x_train)

  print('variancia explicada:', sum(pca.explained_variance_ratio_))
  x_train_pca = pca.transform(x_train)
  x_test_pca = pca.transform(x_test)

  x_train_pca_norm = (x_train_pca - x_train_pca.mean(axis=0))/  x_train_pca.std(axis=0)
  x_test_pca_norm = (x_test_pca - x_train_pca.mean(axis=0))/  x_train_pca.std(axis=0)

  return x_train_pca_norm, x_test_pca_norm

In [62]:
X_train, X_test, y_train_n, y_test_n, y_train, y_test = train_test_separation(X,y)

Shape (70, 4608) (31, 4608) (70, 3) (31, 3) (70,) (31,)


In [66]:
X_train_pca, X_test_pca = apply_PCA(n_pca=50, x_train=X_train, x_test=X_test)

variancia explicada: 0.864794452061688


In [200]:
def unpack_dict_as_str(dicionario):
  string=''

  for k,v in dicionario.items():
    string= string + str(k) + ':[ '
    try:
      for v_ in v:
        string= string + str(v_) + ' '
      string= string+'] '
    except:
      string= string + str(v) + '] '
  return string.strip()

# unpack_dict_as_str({'a':[2,5,6], 'b':[6,7]})

'a:[ 2 5 6 ] b:[ 6 7 ]'

In [363]:
def pipeline_integrated_onedim(nome_do_modelo, #nome do modelo
                               classifier, # classificador
                               param_iteracao, # parametro iteração - deve conter um dicionario com a lista de valores a ser iterado
                               param_fixo,  # parametro fixo e respectivo valor (uma lista)
                               data, # dicionario de treino e teste
                               kf= KFold(n_splits=5, shuffle=True, random_state=42),
                               neural_network=False): #kfold

  print('Pipeline de testes para o modelo:', nome_do_modelo)
  X_train = data['X_train']
  X_test = data['X_test']
  y_train = data['y_train']
  y_test = data['y_test']

  log={}
  log['exec_time'] = 'Execução em: {}'.format(datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S"))
  log['model_name'] = nome_do_modelo
  log['parametro_fixo'] = '{}:[{}]'.format(param_fixo[0], param_fixo[1])
  log['parametro_variavel'] = unpack_dict_as_str(param_iteracao)
  log['kfold'] = kf

  best_acc_val = -1
  best_model=None
  best_param = None
  it=0

  for parametro, valores in param_iteracao.items():
    cv_report = {}
    for valor in valores:
      it+=1

      cv_report['cv_results_'+str(it)] = {}
      print('Execução em: {}'.format(datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")))
      print("====================================================================")
      print('iteração:{}'.format(it))
      param = {parametro:valor}
      param[param_fixo[0]] = param_fixo[1]
      print('Testando parametros:\n', unpack_dict_as_str(param))
      print("====================================================================")
      
      cv_report['cv_results_'+str(it)]['cv_param']= unpack_dict_as_str(param)

      for k_index,(train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
        y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]

        # Ajusta o parâmetro usando o dicionário
        for k,v in param.items():
          setattr(classifier,k,v)
        
        if neural_network:
          classifier.fit(X_train_kf, y_train_kf,[X_val_kf,y_val_kf])
        else:
          classifier.fit(X_train_kf, y_train_kf)

        acc_train = accuracy_score(classifier.predict(X_train_kf), y_train_kf)
        acc_val = accuracy_score(classifier.predict(X_val_kf), y_val_kf)
        acc_test = accuracy_score(classifier.predict(X_test), y_test)

        if acc_val>best_acc_val:
          best_model=classifier
          best_param = param
          k_index_best = k_index
          best_acc_val=acc_val

        cv_report['cv_results_'+str(it)]['acc_train_'+str(k_index)] = acc_train
        cv_report['cv_results_'+str(it)]['acc_val_'+ str(k_index)] = acc_val
        cv_report['cv_results_'+str(it)]['acc_test_'+ str(k_index)] = acc_test

    log['cv_report']= cv_report

    if neural_network:
      best_model.fit(X_train, y_train,[X_test,y_test])
    else:
      best_model.fit(X_train, y_train)
    
    acc_train_best_model = accuracy_score(best_model.predict(X_train), y_train)
    acc_test_best_model = accuracy_score(best_model.predict(X_test), y_test)
    print('Melhor acurácia no validação obtida com os parametros {}. Treino final: {:.4f} Teste:{:.4f}'.format(unpack_dict_as_str(best_param),acc_train_best_model, acc_test_best_model))
    print('Fim da Execução: {}'.format(datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")))
    
    log['melhor_modelo_obj'] = best_model
    log['best_param_k_fold'] = unpack_dict_as_str(best_param)
    log['acc_train_best_model'] = acc_train_best_model
    log['acc_test_best_model'] = acc_test_best_model

    return log

def dictlog_to_flat_table(log):
  log_i = pd.DataFrame(log, index=[norm_text(modelo)])
  cv_log_i = pd.DataFrame(log['cv_report']).T.reset_index().assign(modelo = norm_text(modelo))
  cv_log_i = pd.merge(log_i,cv_log_i, left_index = True , right_on = 'modelo', how='outer').drop(columns='cv_report')
  return cv_log_i

# Teste com Modelos

In [404]:
logs = pd.DataFrame()

Regressão Logística

In [292]:
modelo = 'Regressão Logística Multinomial Passo Fixo'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier= Multinomial_LogReg.MultinomialLogReg(),
                          param_iteracao= {'alpha':[0.01, 0.1]},
                          param_fixo=['theta',0.001],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n} )

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df,ignore_index=True)

Pipeline de testes para o modelo: Regressão Logística Multinomial Passo Fixo
iteração:1
Testando parametros:
 alpha:[ 0.01] theta:[ 0.001]
it:10000, norm_grad:0.008344351234974125, J:0.007027316236154077
it:20000, norm_grad:0.0041879137241205835, J:0.0035336367712077697
it:30000, norm_grad:0.002797060355018867, J:0.002362323989299491
it:40000, norm_grad:0.0021002300275192436, J:0.0017748903991136405
it:50000, norm_grad:0.0016815774042371373, J:0.0014217238925498155
it:60000, norm_grad:0.001402205165488211, J:0.0011859331996768576
it:70000, norm_grad:0.0012025031134775845, J:0.0010173178119877407
it:80000, norm_grad:0.001052635007872087, J:0.0008907381038232871
it:84225, norm_grad:0.0009999897709287967, J:0.0008462645014992518
it:10000, norm_grad:0.008424667500614325, J:0.007157399030622491
it:20000, norm_grad:0.004226631441136499, J:0.003597416782815458
it:30000, norm_grad:0.0028224714926825337, J:0.0024045306052556817
it:40000, norm_grad:0.0021191191609108984, J:0.0018064280554202705


In [405]:
modelo = 'Regressão Logística Multinomial com Bisseção'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier= Multinomial_LogReg.MultinomialLogReg_bissecao(),
                          param_iteracao= {'itmax_bissec':[5, 20]},
                          param_fixo=['theta',0.001],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n} )

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df,ignore_index=True)

Pipeline de testes para o modelo: Regressão Logística Multinomial com Bisseção
Execução em: 21/01/2023 04:20:44
iteração:1
Testando parametros:
 itmax_bissec:[ 5] theta:[ 0.001]
iteracao: 0 norm_grad: 53.2565892185961
it:12, norm_grad:0.00044619934397123213, J:0.00019468690348727166, alpha_b:673.7083913306446, hl:8.735295950524192e-08
iteracao: 0 norm_grad: 61.945637033569874
it:9, norm_grad:0.0005375438180788925, J:0.0003278728346577502, alpha_b:719.8263727496169, hl:1.1528295075647291e-07
iteracao: 0 norm_grad: 61.64541305739746
it:13, norm_grad:0.0009073614509280886, J:0.0004811199277938615, alpha_b:306.36162704160427, hl:5.050721891015396e-07
iteracao: 0 norm_grad: 60.80793198634237
it:8, norm_grad:0.0002654893314528297, J:0.00011095851119750726, alpha_b:2149.744080462463, hl:8.670065273389452e-08
iteracao: 0 norm_grad: 58.2582552462938
it:14, norm_grad:0.0006501196018655858, J:0.0002970014621189593, alpha_b:924.9284140943028, hl:9.737004571300231e-07
Execução em: 21/01/2023 04:20:

# SVM

In [317]:
modelo = 'SVM Classificador Kernel RBF C=1'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier=  SVM_OVR.SVM_one_vs_all(),
                          param_iteracao= {'sigma':[0.01, 0.1,1]},
                          param_fixo=['C',1],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n})

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df,ignore_index=True)

Pipeline de testes para o modelo: SVM Classificador Kernel RBF
iteração:1
Testando parametros:
 sigma:[ 0.01] C:[ 1]


100%|██████████| 3/3 [00:00<00:00, 18.54it/s]


iteração:2
Testando parametros:
 sigma:[ 0.1] C:[ 1]


100%|██████████| 3/3 [00:00<00:00, 16.88it/s]


iteração:3
Testando parametros:
 sigma:[ 1] C:[ 1]


100%|██████████| 3/3 [00:00<00:00, 12.52it/s]


In [328]:
modelo = 'SVM Classificador Kernel RBF C=10'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier=  SVM_OVR.SVM_one_vs_all(),
                          param_iteracao= {'sigma':[0.01, 0.1,1]},
                          param_fixo=['C',10],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n})

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df,ignore_index=True)

Pipeline de testes para o modelo: SVM Classificador Kernel RBF C=10
iteração:1
Testando parametros:
 sigma:[ 0.01] C:[ 10]


100%|██████████| 3/3 [00:00<00:00, 17.52it/s]


iteração:2
Testando parametros:
 sigma:[ 0.1] C:[ 10]


100%|██████████| 3/3 [00:00<00:00, 16.09it/s]


iteração:3
Testando parametros:
 sigma:[ 1] C:[ 10]


100%|██████████| 3/3 [00:00<00:00, 11.66it/s]


In [326]:
modelo = 'SVM Classificador Kernel RBF C=100'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier=  SVM_OVR.SVM_one_vs_all(),
                          param_iteracao= {'sigma':[0.01, 0.1,1]},
                          param_fixo=['C',100],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n})

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df)

Pipeline de testes para o modelo: SVM Classificador Kernel RBF C=100
iteração:1
Testando parametros:
 sigma:[ 0.01] C:[ 100]


100%|██████████| 3/3 [00:00<00:00, 18.56it/s]


iteração:2
Testando parametros:
 sigma:[ 0.1] C:[ 100]


100%|██████████| 3/3 [00:00<00:00, 16.95it/s]


iteração:3
Testando parametros:
 sigma:[ 1] C:[ 100]


100%|██████████| 3/3 [00:00<00:00, 12.41it/s]


# MLP

In [364]:
modelo = 'MultiLayer Perceptron alpha 0.1'

log = pipeline_integrated_onedim(nome_do_modelo=modelo, 
                          classifier=  ANN.RNA(),
                          param_iteracao= {'h':[5,10]},
                          param_fixo=['alpha',0.1],
                          data={'X_train':X_train_pca, 'X_test':X_test_pca, 'y_train':y_train_n, 'y_test':y_test_n},
                          neural_network=True)

log_df = dictlog_to_flat_table(log)
logs = logs.append(log_df,ignore_index=True)

Pipeline de testes para o modelo: MultiLayer Perceptron alpha 0.1
Execução em: 21/01/2023 03:44:45
iteração:1
Testando parametros:
 h:[ 5] alpha:[ 0.1]
nep: 10000 EQM: 0.7009551184557535 EQM_val: 0.63545279749338 norm: 0.0012687082365433209
nep: 20000 EQM: 0.7009551184557535 EQM_val: 0.6284035365766786 norm: 0.0005955645268373699
nep: 30000 EQM: 0.7009551184557535 EQM_val: 0.6287791852130543 norm: 0.00038401351734432336
nep: 40000 EQM: 0.7009551184557535 EQM_val: 0.6289036376732141 norm: 0.00024281642522301926
nep: 50000 EQM: 0.7009551184557535 EQM_val: 0.628600065232641 norm: 0.00018628880218581886
Convergência em 50000 epochs
nep: 10000 EQM: 0.7375571613578742 EQM_val: 0.7073815696484845 norm: 0.0017305623492521853
nep: 20000 EQM: 0.7375571613578742 EQM_val: 0.7156546523993921 norm: 0.0005324970329542368
nep: 30000 EQM: 0.7375571613578742 EQM_val: 0.7180977730292941 norm: 0.0003318819211747358
nep: 40000 EQM: 0.7375571613578742 EQM_val: 0.7188588887578102 norm: 0.0002418468185016222


In [406]:
pd.set_option('display.max_columns', None)
logs.head(3)

,exec_time,model_name,parametro_fixo,parametro_variavel,kfold,melhor_modelo_obj,best_param_k_fold,acc_train_best_model,acc_test_best_model,index,cv_param,acc_train_0,acc_val_0,acc_test_0,acc_train_1,acc_val_1,acc_test_1,acc_train_2,acc_val_2,acc_test_2,acc_train_3,acc_val_3,acc_test_3,acc_train_4,acc_val_4,acc_test_4,modelo
0,Execução em: 21/01/2023 04:20:44,Regressão Logística Multinomial com Bisseção,theta:[0.001],itmax_bissec:[ 5 20 ],"KFold(n_splits=5, random_state=42, shuffle=True)",<Multinomial_LogReg.MultinomialLogReg_bissecao...,itmax_bissec:[ 5] theta:[ 0.001],1.0,0.903226,cv_results_1,itmax_bissec:[ 5] theta:[ 0.001],1.0,0.857143,0.806452,1.0,0.785714,0.967742,1.0,0.714286,0.903226,1.0,0.642857,0.903226,1.0,0.785714,0.870968,regressao_logistica_multinomial_com_bissecao
1,Execução em: 21/01/2023 04:20:44,Regressão Logística Multinomial com Bisseção,theta:[0.001],itmax_bissec:[ 5 20 ],"KFold(n_splits=5, random_state=42, shuffle=True)",<Multinomial_LogReg.MultinomialLogReg_bissecao...,itmax_bissec:[ 5] theta:[ 0.001],1.0,0.903226,cv_results_2,itmax_bissec:[ 20] theta:[ 0.001],1.0,0.785714,0.774194,1.0,0.571429,0.903226,1.0,0.714286,0.870968,1.0,0.857143,0.935484,1.0,0.714286,0.870968,regressao_logistica_multinomial_com_bissecao


# Informações Melhor Modelo

In [415]:
def save_error(label):
  errorFile = open(erro_dir +label + '.txt', 'w') 
  print('Execução em: {}'.format(datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")) , file=errorFile)

  out=''
  for i,r in melhor_modelo_infos.filter(like='acc').T.reset_index().iterrows():
    out=out + str(r['index']) + ':' + str(r[0]) + '\n'
  print(out, file=errorFile)
  errorFile.close()

In [425]:
label = tag + '_' + 'melhor'

melhor_modelo_infos = logs.sort_values(by='acc_test_best_model',ascending=False).head(1)

# Salva mehlhor modelo
melhor_modelo = melhor_modelo_infos['melhor_modelo_obj'].iloc[0]
with open(model_save_dir + label +'.obj', 'wb') as fp:
  pickle.dump(melhor_modelo, fp)

# Salva o erro
save_error(label)

#Salva o config
melhor_modelo_infos.drop(columns = melhor_modelo_infos.filter(like='acc').columns).T.rename(columns={0:'config'}).to_csv(config_dir + label  +'.txt', sep=':')

In [426]:
label = tag + '_' + 'pior'

pior_modelo_infos = logs.sort_values(by='acc_test_best_model',ascending=False).tail(1)

# Salva pior modelo
pior = pior_modelo_infos['melhor_modelo_obj'].iloc[0]
with open(model_save_dir + label +'.obj', 'wb') as fp:
  pickle.dump(pior, fp)

# Salva o erro
save_error(label)

#Salva o config
melhor_modelo_infos.drop(columns = pior_modelo_infos.filter(like='acc').columns).T.rename(columns={0:'config'}).to_csv(config_dir + label  +'.txt', sep=':')